# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import yt_dlp
import time
import os

path_sql = "repository\\querys\\operation\\insert\\video_historic.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_insert = file.read()
   
with open("repository\\querys\\operation\\read\\video.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_videos = db_manager.read_df(sql_query_read)

In [ ]:
parada = 72508
df_filtrado = df_videos.iloc[parada:].copy()
for idx, (i, row) in enumerate(tqdm(df_filtrado.iterrows(), total=df_videos.shape[0] - parada, desc="Processing videos")):
    time.sleep(1)
    video_info = video.get_video_info(url_or_id=row["url"], timeout=10)
    ID_PLAYLIST     = row["id_playlist"] if "id_playlist" in row else None
    TIME_MOVIE      = video_info["duration_seconds"] if "duration_seconds" in video_info else -1
    PUBLISHED_DATE  = video_info["published_date"] if "published_date" in video_info else "" 
    
    sql_template = Template(sql_query_insert)
    rendered_sql = sql_template.render(
        ID_MOVIE            = row["id_movie"],
        TITLE               = video_info["title"].replace("'", "").strip() if "title" in video_info else "",
        DESCRIPTION         = video_info["description"].replace("'", "").strip() if "description" in video_info else None,
        VIEWS               = video_info["views_count"] if "views_count" in video_info else None,
        LIKES               = video_info["likes_count"] if "likes_count" in video_info else None,
        LIVE                = video_info["is_live"] if "is_live" in video_info else None,
        KEYWORDS            = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["keywords"]])+"'" if "keywords" in video_info else None,
        AVAILABLE_COUNTRIES = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["available_countries"]])+"'" if "available_countries" in video_info else None,
        CATEGORY            = video_info["category"] if "category" in video_info else None,
        FAMILY_FRIENDLY     = video_info["family_friendly"] if "family_friendly" in video_info else None,
        DT_UPLOAD           = datetime.fromisoformat(video_info["upload_date_text"]).date().isoformat() if "upload_date_text" in video_info else None, 
        DT_PUBLISH          = datetime.fromisoformat(video_info["publish_date_text"]).date().isoformat() if "publish_date_text" in video_info else None,
        DT_START            = datetime.now().strftime("%Y-%m-%d"),
        DT_END              = '1900-01-01'
    ).replace("'None'", "NULL").replace("None", "NULL")\
     .replace("'False'", "FALSE").replace("False", "FALSE")\
     .replace("'True'", "TRUE").replace("True", "TRUE")

    try:
        db_manager.execute_raw(rendered_sql)
    except Exception as e:
        time.sleep(120)
        print(f"Error inserting video data: {e}")
        print(f"SQL: {rendered_sql}")
        db_manager.connection.rollback()
    

In [ ]:
idx